In [1]:
model_file_path = "/home/gsoykan/Desktop/comp541/comp541_term_project/results/imagenet-resnet-152-dag.mat"
cat_img_url = "https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg"

"https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg"

In [2]:
# https://nextjournal.com/mpd/image-classification-with-knet

In [3]:
import CUDA

In [4]:
using MAT, OffsetArrays, FFTViews, ArgParse, Images, Knet, ImageMagick
include("modular.resnet.jl")

generate_resnet_from_weights (generic function with 1 method)

In [5]:
atype = CUDA.functional() ? KnetArray{Float32} : Array{Float32}
Knet.atype() = atype

In [6]:
function get_params(params, atype)
    len = length(params["value"])
    ws, ms = [], []
    for k = 1:len
        name = params["name"][k]
        value = convert(Array{Float32}, params["value"][k])

        if endswith(name, "moments")
            push!(ms, reshape(value[:,1], (1,1,size(value,1),1)))
            push!(ms, reshape(value[:,2], (1,1,size(value,1),1)))
        elseif startswith(name, "bn")
            push!(ws, reshape(value, (1,1,length(value),1)))
        elseif startswith(name, "fc") && endswith(name, "filter")
            push!(ws, transpose(reshape(value,(size(value,3),size(value,4)))))
        elseif startswith(name, "conv") && endswith(name, "bias")
            push!(ws, reshape(value, (1,1,length(value),1)))
        else
            push!(ws, value)
        end
    end
    map(wi->convert(atype, wi), ws),
    map(mi->convert(atype, mi), ms)
end

get_params (generic function with 1 method)

In [7]:
# From vgg.jl
function data(img, averageImage)
    if occursin("://",img)
        @info "Downloading $img"
        img = download(img)
    end
    a0 = load(img)
    new_size = ntuple(i->div(size(a0,i)*224,minimum(size(a0))),2)
    a1 = Images.imresize(a0, new_size)
    i1 = div(size(a1,1)-224,2)
    j1 = div(size(a1,2)-224,2)
    b1 = a1[i1+1:i1+224,j1+1:j1+224]
    c1 = permutedims(channelview(b1), (3,2,1))
    d1 = convert(Array{Float32}, c1)
    e1 = reshape(d1[:,:,1:3], (224,224,3,1))
    f1 = (255 * e1 .- averageImage)
    g1 = permutedims(f1, [2,1,3,4])
end

data (generic function with 1 method)

In [8]:
# Batch Normalization Layer
# works both for convolutional and fully connected layers
# mode, 0=>train, 1=>test
function batchnorm(w, x, ms; mode=1, epsilon=1e-5)
    mu, sigma = nothing, nothing
    if mode == 0
        d = ndims(x) == 4 ? (1,2,4) : (2,)
        s = prod(size(x,d...))
        mu = sum(x,d) / s
        x0 = x .- mu
        x1 = x0 .* x0
        sigma = sqrt(epsilon + (sum(x1, d)) / s)
    elseif mode == 1
        mu = popfirst!(ms)
        sigma = popfirst!(ms)
    end

    # we need getval in backpropagation
    push!(ms, AutoGrad.value(mu), AutoGrad.value(sigma))
    xhat = (x.-mu) ./ sigma
    return w[1] .* xhat .+ w[2]
end

batchnorm (generic function with 1 method)

In [9]:
function reslayerx0(w,x,ms; padding=0, stride=1, mode=1)
    b  = conv4(w[1],x; padding=padding, stride=stride)
    bx = batchnorm(w[2:3],b,ms; mode=mode)
end

reslayerx0 (generic function with 1 method)

In [10]:
function reslayerx1(w,x,ms; padding=0, stride=1, mode=1)
    relu.(reslayerx0(w,x,ms; padding=padding, stride=stride, mode=mode))
end

reslayerx1 (generic function with 1 method)

In [11]:
function reslayerx2(w,x,ms; pads=[0,1,0], strides=[1,1,1], mode=1)
    ba = reslayerx1(w[1:3],x,ms; padding=pads[1], stride=strides[1], mode=mode)
    bb = reslayerx1(w[4:6],ba,ms; padding=pads[2], stride=strides[2], mode=mode)
    bc = reslayerx0(w[7:9],bb,ms; padding=pads[3], stride=strides[3], mode=mode)
end

reslayerx2 (generic function with 1 method)

In [12]:
function reslayerx3(w,x,ms; pads=[0,0,1,0], strides=[2,2,1,1], mode=1) # 12
    a = reslayerx0(w[1:3],x,ms; stride=strides[1], padding=pads[1], mode=mode)
    b = reslayerx2(w[4:12],x,ms; strides=strides[2:4], pads=pads[2:4], mode=mode)
    relu.(a .+ b)
end

reslayerx3 (generic function with 1 method)

In [13]:
function reslayerx4(w,x,ms; pads=[0,1,0], strides=[1,1,1], mode=1)
    relu.(x .+ reslayerx2(w,x,ms; pads=pads, strides=strides, mode=mode))
end

reslayerx4 (generic function with 1 method)

In [14]:
function reslayerx5(w,x,ms; strides=[2,2,1,1], mode=1)
    x = reslayerx3(w[1:12],x,ms; strides=strides, mode=mode)
    for k = 13:9:length(w)
        x = reslayerx4(w[k:k+8],x,ms; mode=mode)
    end
    return x
end

reslayerx5 (generic function with 1 method)

In [15]:
# mode, 0=>train, 1=>test
function resnet152(w,x,ms; mode=1)
    # layer 1
    conv1 = reslayerx1(w[1:3],x,ms; padding=3, stride=2, mode=mode)
    pool1 = pool(conv1; window=3, stride=2)

    # layer 2,3,4,5
    r2 = reslayerx5(w[4:33], pool1, ms; strides=[1,1,1,1], mode=mode)
    r3 = reslayerx5(w[34:108], r2, ms; mode=mode)
    r4 = reslayerx5(w[109:435], r3, ms; mode=mode)
    r5 = reslayerx5(w[436:465], r4, ms; mode=mode)

    # fully connected layer
    pool5  = pool(r5; stride=1, window=7, mode=2)
    fc1000 = w[466] * mat(pool5) .+ w[467]
end

resnet152 (generic function with 1 method)

In [16]:
o = Dict(
  :atype => KnetArray{Float32},
  :model => model_file_path,
  :image => cat_img_url,
  :top   => 10
)

Dict{Symbol,Any} with 4 entries:
  :atype => KnetArray{Float32,N} where N
  :top   => 10
  :image => "https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrL…
  :model => "/home/gsoykan/Desktop/comp541/comp541_term_project/results/imagene…

In [17]:
function predict(o)
	@info "Reading $(o[:model])"
	model = matread(abspath(o[:model]))
	avgimg = model["meta"]["normalization"]["averageImage"]
	avgimg = convert(Array{Float32}, avgimg)
	description = model["meta"]["classes"]["description"]
	w, ms = get_params(model["params"], o[:atype])

	@info "Reading $(o[:image])"
	img = data(o[:image], avgimg)
	img = convert(o[:atype], img)

	@info "Classifying."
	#@time y1 = resnet152(w,img,ms)
    modular_resnet152 = generate_resnet_from_weights(w, ms)
    y1 = modular_resnet152(img)
  
  return y1, description
end

predict (generic function with 1 method)

In [18]:
model = matread(abspath(o[:model]))
w, ms = get_params(model["params"], o[:atype])


(KnetArray{Float32,N} where N[K32(7,7,3,64)[2.5623413e-7⋯], K32(1,1,64,1)[0.17125425⋯], K32(1,1,64,1)[-0.9211457⋯], K32(1,1,64,256)[1.0009978e-7⋯], K32(1,1,256,1)[1.0431781⋯], K32(1,1,256,1)[-0.42354646⋯], K32(1,1,64,64)[5.7680538e-8⋯], K32(1,1,64,1)[1.3838099⋯], K32(1,1,64,1)[0.2129703⋯], K32(3,3,64,64)[-0.013626201⋯]  …  K32(1,1,512,1)[0.92608565⋯], K32(1,1,512,1)[-0.91251934⋯], K32(3,3,512,512)[-0.004375612⋯], K32(1,1,512,1)[1.0832604⋯], K32(1,1,512,1)[-0.8033409⋯], K32(1,1,512,2048)[-0.015731078⋯], K32(1,1,2048,1)[1.4361827⋯], K32(1,1,2048,1)[-1.7722093⋯], K32(1000,2048)[-0.014174754⋯], K32(1000,1)[-0.01870472⋯]], KnetArray{Float32,4}[K32(1,1,64,1)[-9.028228e-5⋯], K32(1,1,64,1)[0.005939396⋯], K32(1,1,256,1)[-0.764459⋯], K32(1,1,256,1)[0.8341639⋯], K32(1,1,64,1)[-2.0849948⋯], K32(1,1,64,1)[1.1826472⋯], K32(1,1,64,1)[-1.1165951⋯], K32(1,1,64,1)[1.0983644⋯], K32(1,1,256,1)[-0.1807499⋯], K32(1,1,256,1)[0.2596636⋯]  …  K32(1,1,512,1)[-0.39513633⋯], K32(1,1,512,1)[0.3477594⋯], K32(1,1,20

In [19]:
println(size(w[1:3]))
w[2:3]

(3,)


2-element Array{KnetArray{Float32,N} where N,1}:
 K32(1,1,64,1)[0.17125425⋯]
 K32(1,1,64,1)[-0.9211457⋯]

In [20]:
Knet.atype()

KnetArray{Float32,N} where N

In [21]:
y1, description = predict(o)

┌ Info: Reading /home/gsoykan/Desktop/comp541/comp541_term_project/results/imagenet-resnet-152-dag.mat
└ @ Main In[17]:2
┌ Info: Reading https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg
└ @ Main In[17]:9
┌ Info: Downloading https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg
└ @ Main In[7]:4
┌ Info: Classifying.
└ @ Main In[17]:13


(K32(1000,1)[-1.847644⋯], Any["tench, Tinca tinca" "goldfish, Carassius auratus" … "ear, spike, capitulum" "toilet tissue, toilet paper, bathroom tissue"])

In [22]:
z1 = vec(Array(y1))
s1 = sortperm(z1,rev=true)
p1 = exp.(logp(z1))

1000-element Array{Float32,1}:
 1.6237807f-6
 1.7950113f-6
 3.0487963f-6
 1.8512f-6
 2.2960382f-6
 9.160164f-5
 8.271447f-5
 4.423799f-5
 0.0016335086
 4.4700588f-5
 4.778216f-5
 1.0584847f-5
 0.00033455592
 ⋮
 0.00027946424
 7.1559484f-6
 7.852398f-5
 8.843055f-7
 2.2513111f-6
 3.4196191f-6
 5.8417627f-6
 8.232677f-6
 4.2582205f-6
 1.3455667f-5
 0.00015772587
 0.0008228442

In [23]:
using Printf

for ind in s1[1:o[:top]]
  print("$(description[ind]): $(@sprintf("%.2f",p1[ind]*100))%\n")
end

tabby, tabby cat: 32.22%
Egyptian cat: 22.96%
tiger cat: 9.38%
mongoose: 4.29%
lynx, catamount: 1.95%
bath towel: 1.41%
prayer rug, prayer mat: 1.28%
teddy, teddy bear: 1.24%
great grey owl, great gray owl, Strix nebulosa: 1.03%
guinea pig, Cavia cobaya: 1.00%
